In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
import pickle

In [24]:
# Load the trained model
model = load_model('model.keras')

# Load the encoder
with open("encoders.pkl", "rb") as file:
    encoders = pickle.load(file)

gender_encoder = encoders["gender_encoder"]
geo_encoder = encoders["geo_encoder"]

# Load the scaler
with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [25]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000,
    'Gender': 'Male',
    'Geography': 'France',
}
# Convert into DataFrame
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender,Geography
0,600,40,3,60000,2,1,1,50000,Male,France


In [26]:
# Transform Gender
input_df['Gender'] = gender_encoder.transform(input_df['Gender'])

# Transform Geography
geo_encoded = geo_encoder.transform(input_df[['Geography']])
geo_df = pd.DataFrame(geo_encoded, columns=geo_encoder.get_feature_names_out(['Geography']), index=input_df.index)

# Combine everything
input_df = pd.concat([input_df.drop('Geography', axis=1), geo_df], axis=1)
input_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender,Geography_France,Geography_Germany,Geography_Spain
0,600,40,3,60000,2,1,1,50000,1,1.0,0.0,0.0


In [27]:
# Scaling the input data
input_scaled = scaler.fit_transform(input_df)
input_scaled

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [28]:
# Predict Churn
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step


array([[0.1832902]], dtype=float32)

In [29]:
prediction_probability = prediction[0][0]
print(f"Prediction Probability is : ",prediction_probability)

Prediction Probability is :  0.1832902


In [31]:
if prediction_probability > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is not likely to churn.
